<a href="https://colab.research.google.com/github/dtobi59/nsga3-mammography-hpo-main/blob/master/colab_mammography_hpo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NSGA-III Hyperparameter Optimization for Mammography Classification

Multi-objective hyperparameter optimization for deep learning models in breast cancer detection.

**Objectives:**
- Maximize: Sensitivity, Specificity, AUC
- Minimize: Model Size

## 1. Setup Environment

In [1]:
# Check GPU availability
!nvidia-smi

Fri Dec 19 02:12:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 2. Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 3. Clone Repository & Install Dependencies

In [3]:
# Clone the repository
!git clone https://github.com/dtobi59/nsga3-mammography-hpo-main.git
%cd nsga3-mammography-hpo-main

Cloning into 'nsga3-mammography-hpo-main'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 43 (delta 17), reused 39 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (43/43), 50.58 KiB | 12.64 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/content/nsga3-mammography-hpo-main


In [4]:
# Install required packages
!pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 8.1 MB/s eta 0:00:00


## 4. Configure Dataset Path

Update the path below to point to your dataset on Google Drive.

**Supported datasets:**
- `vindr` - VinDr-Mammo
- `inbreast` - INbreast

**Expected structure:**

**VinDr-Mammo:**
```
vindr-mammo/
├── images/
│   └── {study_id}/
│       └── {image_id}.dicom
└── metadata/
    └── breast-level_annotations.csv
```

**INbreast:**
```
inbreast/
└── INbreast Release 1.0/
    ├── AllDICOMs/
    │   └── {id}_{hash}_MG_{L/R}_{CC/MLO}_ANON.dcm
    └── INbreast.csv
```

In [5]:
# Configure your dataset
DATASET_NAME = "vindr"  # or "inbreast"
DATA_ROOT = "/content/drive/MyDrive/vindr-mammo"  # Update this path!
OUTPUT_DIR = "/content/drive/MyDrive/nsga3_outputs"  # Where to save results

## 5. Load Dataset

In [ ]:
from dataset import prepare_dataset

print(f"Loading {DATASET_NAME} dataset from {DATA_ROOT}...")

train_paths, train_labels, val_paths, val_labels = prepare_dataset(
    dataset_name=DATASET_NAME,
    data_root=DATA_ROOT
)

print(f"\nDataset loaded successfully!")
print(f"Train samples: {len(train_paths)}")
print(f"Validation samples: {len(val_paths)}")
print(f"Train labels distribution: {sum(train_labels)} malignant, {len(train_labels) - sum(train_labels)} benign")
print(f"Val labels distribution: {sum(val_labels)} malignant, {len(val_labels) - sum(val_labels)} benign")

Loading vindr dataset from /content/drive/MyDrive/vindr-mammo...
VinDr-Mammo: /content/drive/MyDrive/vindr-mammo
  Loaded 20000 rows from CSV
  BI-RADS distribution in downloaded images:
    BI-RADS 1: 13406
    BI-RADS 2: 4676
    BI-RADS 3: 930
    BI-RADS 4: 762
    BI-RADS 5: 226
  After BI-RADS filter: 19070 (Benign: 18082, Malignant: 988)


Scanning:  89%|████████▉ | 16937/19070 [05:00<01:14, 28.54it/s]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from dataset import load_dicom_image

def visualize_samples(image_paths, labels, dataset_name, n_samples=6):
    """
    Visualize sample mammography images.

    Args:
        image_paths: List of image file paths
        labels: List of labels (0=benign, 1=malignant)
        dataset_name: Name for the plot title (e.g., 'Training', 'Validation')
        n_samples: Number of samples to display
    """
    # Select random samples
    n_samples = min(n_samples, len(image_paths))
    indices = np.random.choice(len(image_paths), n_samples, replace=False)

    # Create subplot grid
    n_cols = 3
    n_rows = (n_samples + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))

    if n_rows == 1:
        axes = axes.reshape(1, -1)

    fig.suptitle(f'{dataset_name} Dataset Samples', fontsize=16, fontweight='bold')

    for idx, ax_idx in enumerate(range(n_samples)):
        row = ax_idx // n_cols
        col = ax_idx % n_cols
        ax = axes[row, col]

        sample_idx = indices[idx]
        img_path = image_paths[sample_idx]
        label = labels[sample_idx]

        try:
            # Load DICOM image
            image = load_dicom_image(img_path, apply_clahe=True)

            # Display
            ax.imshow(image, cmap='gray')
            label_text = 'MALIGNANT' if label == 1 else 'BENIGN'
            label_color = 'red' if label == 1 else 'green'
            ax.set_title(f'{label_text}', fontsize=12, fontweight='bold', color=label_color)
            ax.axis('off')

            # Add filename as subtitle
            filename = os.path.basename(img_path)
            ax.text(0.5, -0.05, filename[:30] + '...' if len(filename) > 30 else filename,
                   ha='center', va='top', transform=ax.transAxes,
                   fontsize=8, style='italic')

        except Exception as e:
            ax.text(0.5, 0.5, f'Error loading image:\n{str(e)[:50]}',
                   ha='center', va='center', transform=ax.transAxes)
            ax.axis('off')

    # Hide extra subplots
    for idx in range(n_samples, n_rows * n_cols):
        row = idx // n_cols
        col = idx % n_cols
        axes[row, col].axis('off')

    plt.tight_layout()
    plt.show()

# Visualize training samples
print("=" * 70)
print("TRAINING SET SAMPLES")
print("=" * 70)
visualize_samples(train_paths, train_labels, 'Training', n_samples=6)

print("\n" + "=" * 70)
print("VALIDATION SET SAMPLES")
print("=" * 70)
visualize_samples(val_paths, val_labels, 'Validation', n_samples=6)

## 5.5 Visualize Sample Images

Let's look at some sample mammography images from the training and validation sets.

## 6. Configure Optimization

Adjust these parameters based on your computational budget:
- **pop_size**: Population size (more = better exploration, longer runtime)
- **n_generations**: Number of generations (more = better convergence)
- **epochs**: Training epochs per evaluation (reduce for faster testing)

In [ ]:
from config import ExperimentConfig
from dataclasses import asdict

# Load default config
config = ExperimentConfig()

# Customize NSGA-III parameters
config.nsga3.pop_size = 20        # Recommended: 20-50 for quick runs, 100+ for thorough search
config.nsga3.n_generations = 10   # Recommended: 5-10 for testing, 20-50 for production

# Training epochs per evaluation (reduce for faster testing)
TRAINING_EPOCHS = 5  # Recommended: 5 for testing, 10-20 for production

print(f"Optimization Configuration:")
print(f"  Population size: {config.nsga3.pop_size}")
print(f"  Generations: {config.nsga3.n_generations}")
print(f"  Epochs per evaluation: {TRAINING_EPOCHS}")
print(f"  Total evaluations: ~{config.nsga3.pop_size * config.nsga3.n_generations}")
print(f"\nSearching hyperparameters:")

# Convert dataclass to dictionary for iteration
hp_space_dict = asdict(config.hyperparameter_space)
for param, values in hp_space_dict.items():
    print(f"  {param}: {values}")

## 7. Create Evaluation Function

In [ ]:
from training import full_evaluation

def make_eval_fn(tp, tl, vp, vl, epochs=5):
    """
    Creates an evaluation function for the optimizer.

    Args:
        tp: Training paths
        tl: Training labels
        vp: Validation paths
        vl: Validation labels
        epochs: Number of training epochs

    Returns:
        Evaluation function that takes hyperparameter config and returns objectives
    """
    def eval_fn(hp_config):
        hp_config = hp_config.copy()
        hp_config['epochs'] = epochs
        return full_evaluation(
            hp_config,
            tp, tl, vp, vl,
            device='cuda',  # Use GPU
            verbose=True
        )
    return eval_fn

eval_function = make_eval_fn(
    train_paths, train_labels,
    val_paths, val_labels,
    epochs=TRAINING_EPOCHS
)

print("Evaluation function created successfully!")

## 8. Run Optimization

This will take some time depending on your configuration.

**Estimated runtime:**
- Small (pop=20, gen=5, epochs=5): ~30-60 minutes
- Medium (pop=50, gen=10, epochs=10): ~2-4 hours
- Large (pop=100, gen=20, epochs=20): ~8-12 hours

In [ ]:
from optimization import run_optimization
import time

# Set random seed for reproducibility
# Change this value (e.g., 42, 123, 456) for different runs
RANDOM_SEED = 42

print(f"Starting optimization with seed={RANDOM_SEED}...\n")
start_time = time.time()

results = run_optimization(
    hp_space=config.hyperparameter_space,
    nsga_config=config.nsga3,
    eval_function=eval_function,
    output_dir=OUTPUT_DIR,
    seed=RANDOM_SEED  # Set seed here
)

elapsed_time = time.time() - start_time
print(f"\n{'='*70}")
print(f"Optimization completed in {elapsed_time/3600:.2f} hours")
print(f"{'='*70}")

## 9. View Results

In [ ]:
# Display Pareto front solutions
print(f"\nFound {len(results['pareto_configs'])} Pareto-optimal solutions\n")
print("="*100)

for i, (cfg, obj) in enumerate(zip(results['pareto_configs'], results['pareto_F'])):
    print(f"\nSolution {i+1}:")
    print("-" * 100)

    # Architecture details
    print(f"  Architecture:")
    print(f"    Backbone: {cfg['backbone']}")
    print(f"    Unfreeze: {cfg['unfreeze_strategy']}")
    print(f"    Dropout: {cfg['dropout_rate']:.2f}")
    print(f"    FC Hidden: {cfg['fc_hidden_size']}")

    # Training details
    print(f"  Training:")
    print(f"    Optimizer: {cfg['optimizer']}")
    print(f"    Learning Rate: {cfg['learning_rate']:.6f}")
    print(f"    Batch Size: {cfg['batch_size']}")
    print(f"    Loss: {cfg['loss_function']}")
    if cfg['loss_function'] == 'focal':
        print(f"    Focal Gamma: {cfg.get('focal_gamma', 2.0):.2f}")

    # Augmentation
    print(f"  Augmentation:")
    print(f"    Horizontal Flip: {cfg['horizontal_flip']}")
    print(f"    Rotation: {cfg['rotation_range']:.1f}°")
    print(f"    Mixup: {cfg['use_mixup']}")

    # Performance metrics
    print(f"  Performance:")
    print(f"    Sensitivity: {obj[0]:.4f}")
    print(f"    Specificity: {obj[1]:.4f}")
    print(f"    AUC: {obj[2]:.4f}")
    print(f"    Model Size: {obj[3]:.2f}M parameters")

## 10. Visualize Pareto Front (Optional)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract objectives
pareto_F = np.array(results['pareto_F'])

# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Pareto Front - Trade-off Analysis', fontsize=16, fontweight='bold')

# Sensitivity vs Specificity
axes[0, 0].scatter(pareto_F[:, 0], pareto_F[:, 1], c='blue', s=100, alpha=0.6)
axes[0, 0].set_xlabel('Sensitivity', fontsize=12)
axes[0, 0].set_ylabel('Specificity', fontsize=12)
axes[0, 0].set_title('Sensitivity vs Specificity')
axes[0, 0].grid(True, alpha=0.3)

# AUC vs Model Size
axes[0, 1].scatter(pareto_F[:, 2], pareto_F[:, 3], c='green', s=100, alpha=0.6)
axes[0, 1].set_xlabel('AUC', fontsize=12)
axes[0, 1].set_ylabel('Model Size (M params)', fontsize=12)
axes[0, 1].set_title('AUC vs Model Size')
axes[0, 1].grid(True, alpha=0.3)

# Sensitivity vs Model Size
axes[1, 0].scatter(pareto_F[:, 0], pareto_F[:, 3], c='purple', s=100, alpha=0.6)
axes[1, 0].set_xlabel('Sensitivity', fontsize=12)
axes[1, 0].set_ylabel('Model Size (M params)', fontsize=12)
axes[1, 0].set_title('Sensitivity vs Model Size')
axes[1, 0].grid(True, alpha=0.3)

# Overall metrics distribution
metrics = ['Sensitivity', 'Specificity', 'AUC', 'Size (M)']
avg_values = pareto_F.mean(axis=0)
axes[1, 1].bar(metrics, avg_values, color=['blue', 'green', 'red', 'purple'], alpha=0.6)
axes[1, 1].set_ylabel('Average Value', fontsize=12)
axes[1, 1].set_title('Average Objective Values')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/pareto_front_visualization.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\nVisualization saved to {OUTPUT_DIR}/pareto_front_visualization.png")

## 11. Save Best Configuration for Deployment

In [ ]:
import json

# Find solution with best AUC
best_auc_idx = np.argmax(pareto_F[:, 2])
best_auc_config = results['pareto_configs'][best_auc_idx]
best_auc_objectives = pareto_F[best_auc_idx]

print(f"Best AUC Configuration:")
print(f"  AUC: {best_auc_objectives[2]:.4f}")
print(f"  Sensitivity: {best_auc_objectives[0]:.4f}")
print(f"  Specificity: {best_auc_objectives[1]:.4f}")
print(f"\nConfiguration: {json.dumps(best_auc_config, indent=2)}")

# Save to file
with open(f"{OUTPUT_DIR}/best_auc_config.json", 'w') as f:
    json.dump({
        'config': best_auc_config,
        'objectives': {
            'sensitivity': float(best_auc_objectives[0]),
            'specificity': float(best_auc_objectives[1]),
            'auc': float(best_auc_objectives[2]),
            'model_size_M': float(best_auc_objectives[3])
        }
    }, f, indent=2)

print(f"\nBest configuration saved to {OUTPUT_DIR}/best_auc_config.json")

## 12. Select Configuration by Preference (Optional)

Choose a configuration based on your priorities.

In [ ]:
# Find different optimal solutions
best_sensitivity_idx = np.argmax(pareto_F[:, 0])
best_specificity_idx = np.argmax(pareto_F[:, 1])
smallest_model_idx = np.argmin(pareto_F[:, 3])

print("Different Optimization Preferences:\n")

print(f"1. Best Sensitivity: {pareto_F[best_sensitivity_idx, 0]:.4f}")
print(f"   AUC: {pareto_F[best_sensitivity_idx, 2]:.4f}, Size: {pareto_F[best_sensitivity_idx, 3]:.2f}M\n")

print(f"2. Best Specificity: {pareto_F[best_specificity_idx, 1]:.4f}")
print(f"   AUC: {pareto_F[best_specificity_idx, 2]:.4f}, Size: {pareto_F[best_specificity_idx, 3]:.2f}M\n")

print(f"3. Best AUC: {pareto_F[best_auc_idx, 2]:.4f}")
print(f"   Sens: {pareto_F[best_auc_idx, 0]:.4f}, Spec: {pareto_F[best_auc_idx, 1]:.4f}\n")

print(f"4. Smallest Model: {pareto_F[smallest_model_idx, 3]:.2f}M")
print(f"   AUC: {pareto_F[smallest_model_idx, 2]:.4f}, Sens: {pareto_F[smallest_model_idx, 0]:.4f}, Spec: {pareto_F[smallest_model_idx, 1]:.4f}")

## 13. Results Summary

All results have been saved to your Google Drive:
- Pareto-optimal configurations
- Optimization history
- Visualizations
- Best configuration JSON

You can now:
1. Download the results from Google Drive
2. Use the best configuration to train a final model
3. Deploy the model for inference
4. Run additional experiments with different parameters

In [ ]:
# Example: Run optimization with 3 different seeds
# Uncomment and run this cell for multiple experiments

# seeds = [42, 123, 456]
# all_results = {}

# for seed in seeds:
#     print(f"\n{'='*70}")
#     print(f"Running experiment with seed={seed}")
#     print(f"{'='*70}\n")

#     output_dir_seed = f"{OUTPUT_DIR}/seed_{seed}"

#     results = run_optimization(
#         hp_space=config.hyperparameter_space,
#         nsga_config=config.nsga3,
#         eval_function=eval_function,
#         output_dir=output_dir_seed,
#         seed=seed
#     )

#     all_results[seed] = results
#     print(f"\nSeed {seed} completed. Found {len(results['pareto_configs'])} Pareto solutions.")

# # Compare results across seeds
# print(f"\n{'='*70}")
# print("SUMMARY ACROSS ALL SEEDS")
# print(f"{'='*70}\n")

# for seed, results in all_results.items():
#     pareto_F = np.array(results['pareto_F'])
#     print(f"Seed {seed}:")
#     print(f"  Pareto solutions: {len(results['pareto_configs'])}")
#     print(f"  Best AUC: {pareto_F[:, 2].max():.4f}")
#     print(f"  Best Sensitivity: {pareto_F[:, 0].max():.4f}")
#     print(f"  Best Specificity: {pareto_F[:, 1].max():.4f}\n")

## 14. Running Multiple Experiments with Different Seeds (Optional)

For robust research results, run the optimization multiple times with different random seeds and compare the Pareto fronts.